In [1]:
# Дополнительные библиотеки

# Графическое отображение различных динамограмм
%matplotlib widget
from matplotlib import pyplot as plt
from matplotlib import gridspec
from ipywidgets import Dropdown
from IPython.display import display
from ipywidgets import interact, IntSlider, widgets, FloatSlider
from itables import show
import numpy as np
import pandas as pd


# Core modules
from DataLoader import *
from DinoStruct import *
from const import *
from Interpolation import *
from MakeDino import *

# Генерация пути до данных 
def GenPath(base, num):
    return base + str(num) + ".txt"

def GenerateSrcPathData(base, start, end):
    return [GenPath(base, i) for i in range(start, end+1)]
    

     Нагрузка [ft]  Перемещение [inch]  Время [c]
0          1718.66                0.00       0.00
1          1708.23                0.00       0.08
2          1698.70                0.01       0.17
3          1691.90                0.01       0.25
4          1698.25                0.02       0.33
..             ...                 ...        ...
195        2033.45                0.02      16.29
196        1901.91                0.01      16.37
197        1757.67                0.01      16.45
198        1721.84                0.00      16.54
199        1718.66                0.00      16.62

[200 rows x 3 columns]


In [10]:
#  Генерация тестов для МК
import re
# Выведем данные в с формате массива

base_filename = "../tmp_data/data"

f = open("test.txt", 'w+')

def GenTest(TestNum):
    filename = base_filename + str(TestNum) + ".txt"
    Orig_dat:OriginalDinoData = LoadOriginalDinoData(filename, False, 0)
    f.write("A2I_DinoForm_TestCase Test{0}= \n".format(TestNum))
    f.write("{\n")
    f_str = "   .in_Load = {"
    for e in Orig_dat.Load:
        f_str += str(e) + "," 
    f_str = re.sub(",$", "},\n", f_str)
    f.write(f_str)

    f_str = "   .in_TimeGrid  = {"
    for e in Orig_dat.TimeIntervals:
        f_str += str(e) + "," 
    f_str = re.sub(",$", "},\n", f_str)
    f.write(f_str)
    
    f_str = "   .in_size  = {0},\n".format(Orig_dat.DataSize)
    f.write(f_str)

    # Строим динамограмму 
    Dino:MakeDino =  MakeDino(filename, SKParam(), 1, 0.003, 0.003, True, 1)
    
    f_str = "   .out_Load = {"
    for e in Dino.DinoData.Load:
        f_str += "{0:0.4f},".format(e*KG2FT) 
    f_str = re.sub(",$", "},\n", f_str)
    f.write(f_str)
    
    
    f_str = "   .out_Position = {"
    for e in Dino.DinoData.PolishRoadMovement:
        f_str += "{0:0.4f},".format(e) 
    f_str = re.sub(",$", "}\n", f_str)
    f.write(f_str)
    f.write("};\n\n")


for i in range(1, 5):
    GenTest(i)

f.close()

In [3]:
# Блок настройки выбора исходных данных для динамограмм
base_path_from = 0
base_path_idx = [[1, 4]]
base_paths = ["../tmp_data/data"]
base_path = base_paths[base_path_from]
start_mydino = base_path_idx[base_path_from][0]
end_mydino = base_path_idx[base_path_from][1]


# Блок настройки выбора исходных данных динамограмм
base_path_from_dino = 0
base_path_idx_dino = [[1,4], [1,4]]

base_path_dinos = ["../tmp_data/dino", "../tmp_data/dino_lufkin"]
base_path_dino = base_path_dinos[base_path_from_dino]
start_dino = base_path_idx_dino[base_path_from_dino][0]
end_dino = base_path_idx_dino[base_path_from_dino][1]



In [4]:
def UI_LoadMyDinos(start, end,PolishRoadMathModel = 2 ,LoadErrorCoef = 0.03, PolishRoadMovementErrorCoef = 0.0001):

    Dinos = [MakeDino(base_path_i, SKParam(), PolishRoadMathModel, LoadErrorCoef, PolishRoadMovementErrorCoef, True, 2) for base_path_i in GenerateSrcPathData(base_path, start,end)]
    return Dinos

def UI_LoadDinos(start, end):
    DinosUsov_Lafkin = [LoadDinoData(base_path_i, 2, 2) for base_path_i in GenerateSrcPathData(base_path_dino, start, end)]
    return DinosUsov_Lafkin

# Подгрузил свои динамограммы и сформировал их 
LoadErrorCoef = 0.03 # Уровень ошибки для динамограммы
PolishRoadMovementErrorCoef = 0.001
PolishRoadMathModel = 1
Dinos =  UI_LoadMyDinos(start_mydino, end_mydino, PolishRoadMathModel, LoadErrorCoef, PolishRoadMovementErrorCoef)

# Загрузка уже готовых динамограмм от лафкина или Усова
DinosUsov_Lafkin = UI_LoadDinos(start_dino, end_dino)

#for Din in DinosUsov_Lafkin:
# # #     Din.PolishRoadMovement.reverse()
    #Din.PolishRoadMovement = Din.PolishRoadMovement[-3:] + Din.PolishRoadMovement[:-3]


def TableShow(idx):
    show(Dinos[idx].DinoData.MyDinoData2DinoData().TablePrint())

def TableShowUsovLafkin(idx):
    show(DinosUsov_Lafkin[idx].TablePrint())



In [5]:
base_path_dino = base_path_dinos[1]
def UI_LoadDinos(start, end):
    D = [LoadDinoData(base_path_i, 2, 2) for base_path_i in GenerateSrcPathData(base_path_dino, start, end)]
    return D

Dinos_Lafkin = UI_LoadDinos(start_dino, end_dino)
for Din in Dinos_Lafkin:
    #Din.RavTimeGrid = [t + 0.20 for t in Din.RavTimeGrid]
    Din.Load = [1.1*l - 0.1 for l in Din.Load]
    
    Din.Load = Din.Load[2:] + Din.Load[:2]
    Din.PolishRoadMovement = Din.PolishRoadMovement[3:] + Din.PolishRoadMovement[:3]


def TableShowLafkin(idx):
    show(Dinos_Lafkin[idx].TablePrint())





In [6]:
# Таблица усилий и перемещений 

print("Мои динамограммы \nbase path = ", base_path)
print("Загрузили динамограммы с ", start_mydino, " по ", end_mydino)
interact(TableShow, idx = (start_mydino-1, end_mydino-1, 1))

print("Динамограммы Усова\nbase path = ", base_path_dino)
print("Загрузили динамограммы с ", start_dino, " по ", end_dino)
interact(TableShowUsovLafkin, idx = (start_dino-1, end_dino-1, 1))


print("Динамограммы Лафкин\nbase path = ", base_path_dino)
print("Загрузили динамограммы с ", start_dino, " по ", end_dino)
interact(TableShowLafkin, idx = (start_dino-1, end_dino-1, 1))


Мои динамограммы 
base path =  ../tmp_data/data
Загрузили динамограммы с  1  по  4


interactive(children=(IntSlider(value=1, description='idx', max=3), Output()), _dom_classes=('widget-interact'…

Динамограммы Усова
base path =  ../tmp_data/dino_lufkin
Загрузили динамограммы с  1  по  4


interactive(children=(IntSlider(value=1, description='idx', max=3), Output()), _dom_classes=('widget-interact'…

Динамограммы Лафкин
base path =  ../tmp_data/dino_lufkin
Загрузили динамограммы с  1  по  4


interactive(children=(IntSlider(value=1, description='idx', max=3), Output()), _dom_classes=('widget-interact'…

<function __main__.TableShowLafkin(idx)>

In [7]:
def PrintDinoMyAndUsov(_start_mydino, _end_mydino, start_u_l_dino, end_u_l_din, title="Usov Data"):

    fig = plt.figure()
    
    # to change size of subplot's
    # set height of each subplot as 8
    fig.set_figheight(7)
    
    # set width of each subplot as 8
    fig.set_figwidth(15)
    
    # create grid for different subplots
    spec = gridspec.GridSpec(ncols=1, nrows=1, wspace=1.0, hspace=1.0)
    ax = fig.add_subplot(spec[0])
    
    ax.set_title("Динамограммы")
    ax.set_xlabel("Перемещение нормированное")
    ax.set_ylabel("Усилие, нормированное")
    line1, line2,line3 = None, None, None
    
    for i in range(_start_mydino, _end_mydino):
        Dino = Dinos[i]
        line1, = ax.plot(Dino.DinoData.PolishRoadMovement, Dino.DinoData.Load, color='red')
        #print("My dino from file: ", i+1, ": ",Dino.DinoData.Load)

    for i in range(start_u_l_dino, end_u_l_din):
        line2, = ax.plot(DinosUsov_Lafkin[i].PolishRoadMovement, DinosUsov_Lafkin[i].Load, color='blue')
    
    
    for i in range(start_u_l_dino, end_u_l_din):
        Dinos_Lafkin[i].PolishRoadMovement.reverse()
        Dinos_Lafkin[i].Load.reverse()
        line3, = ax.plot(Dinos_Lafkin[i].PolishRoadMovement, Dinos_Lafkin[i].Load, color='green')
    

    ax.legend([line1, line2, line3], ["Динамограмма построенная по Схеме Эйтейкена","Динамограмма " + title, "Динамограмма Lufkin"])
    plt.show()


def PrintMyFPolRoadandUsovLafkinFPolRoad(_start_mydino, _end_mydino, start_u_l_dino, end_u_l_din, title="Usov Data"):
    # create a figure
    fig = plt.figure()
    
    # to change size of subplot's
    # set height of each subplot as 8
    fig.set_figheight(7)
    
    # set width of each subplot as 8
    fig.set_figwidth(15)
    
    # create grid for different subplots
    spec = gridspec.GridSpec(ncols=1, nrows=2, wspace=1.0, hspace=1.0)
    
    line1, line2,line3 = None, None, None
    # initializing x,y axis value

    ax0 = fig.add_subplot(spec[0])
    ax1 = fig.add_subplot(spec[1])

    # Усилие сравниваем 
    for i in range(_start_mydino, _end_mydino):
        Dino = Dinos[i]
        
        line1, = ax0.plot(Dino.DinoData.RavTimeGrid, Dino.DinoData.Load, color='red')

        #arr = [Dino._CalcF(Dino.DinoData.OrigData.TimeGrid[i])[0] - Dino.DinoData.OrigData.Load[i] for i in range(0, len(Dino.DinoData.OrigData.Load))]
        
        #for a in arr:
        #    print('{0:.16f}'.format(a))
        #line3, =  ax0.plot(Dino.DinoData.OrigData.TimeGrid, Dino.DinoData.OrigData.Load , 'go')

    for i in range(start_u_l_dino, end_u_l_din):
        line2, = ax0.plot(DinosUsov_Lafkin[i].RavTimeGrid, DinosUsov_Lafkin[i].Load, color='blue')
    
    for i in range(start_u_l_dino, end_u_l_din):
       
        line3, = ax0.plot(Dinos_Lafkin[i].RavTimeGrid, Dinos_Lafkin[i].Load, color='green')
    
    

    ax0.legend([line1, line2, line3], ["Усилие построенная по Схеме Эйтейкена","Усилие из динамограмм " + title, "Усилие из динамограмм Lufkin"])
    ax0.set_xlabel("Время [c]")
    ax0.set_ylabel("Усилие нормированное")

    # Перемещение 
    for i in range(_start_mydino, _end_mydino):
        Dino = Dinos[i]
        
        line1, = ax1.plot(Dino.DinoData.RavTimeGrid, Dino.DinoData.PolishRoadMovement, color='red')

    for i in range(start_u_l_dino, end_u_l_din):
        line2, = ax1.plot(DinosUsov_Lafkin[i].RavTimeGrid, DinosUsov_Lafkin[i].PolishRoadMovement, color='blue')
    
    for i in range(start_u_l_dino, end_u_l_din):
        
        line3, = ax1.plot(Dinos_Lafkin[i].RavTimeGrid, Dinos_Lafkin[i].PolishRoadMovement, color='green')
    

    ax1.legend([line1, line2, line3], ["Перемещение построенная по Схеме Эйтейкена","Перемещение из динамограмм " + title, "Перемещение из динамограмм Lufkin"])
    ax1.set_xlabel("Время [c]")
    ax1.set_ylabel("Перемещение нормированное")
    
    plt.show()



In [8]:
# Отобразим наложенные динамограммы мои/Усова(Лафкин)
print("Отобразим наложенные динамограммы мои/Усова(Лафкин)\n")
print("Данные взяты из папки ", base_path, )
print("Данные для динамограмм из папки ", base_path_dino)
interact(PrintDinoMyAndUsov,_start_mydino = IntSlider(value=0,min=start_mydino-1, max=end_mydino-1, step=1),
                                            _end_mydino=IntSlider(value=4,min=start_mydino-1, max=end_mydino, step=1),
                                            start_u_l_dino = IntSlider(value=0,min=start_dino-1, max=end_dino-1, step=1),
                                            end_u_l_din= IntSlider(value=4,min=start_dino-1, max=end_dino, step=1),
                                            title = Dropdown(options=[('Усова'), ('Lafkin')], value='Усова'))


# Сравнение графиков усилия мои/Усова(Лафкин) и графиков перемещения мои/Усова(Лафкин)
print("Отобразим наложенные усилия и перемещения мои/Усова(Лафкин)\n")
print("Данные взяты из папки ", base_path, )
print("Данные для динамограмм из папки ", base_path_dino)
interact(PrintMyFPolRoadandUsovLafkinFPolRoad,_start_mydino = IntSlider(value=0,min=start_mydino-1, max=end_mydino-1, step=1),
                                            _end_mydino=IntSlider(value=4,min=start_mydino-1, max=end_mydino, step=1),
                                            start_u_l_dino = IntSlider(value=0,min=start_dino-1, max=end_dino-1, step=1),
                                            end_u_l_din= IntSlider(value=4,min=start_dino-1, max=end_dino, step=1),
                                            title = Dropdown(options=[('Усова'), ('Lafkin')], value='Усова'))

Отобразим наложенные динамограммы мои/Усова(Лафкин)

Данные взяты из папки  ../tmp_data/data
Данные для динамограмм из папки  ../tmp_data/dino_lufkin


interactive(children=(IntSlider(value=0, description='_start_mydino', max=3), IntSlider(value=4, description='…

Отобразим наложенные усилия и перемещения мои/Усова(Лафкин)

Данные взяты из папки  ../tmp_data/data
Данные для динамограмм из папки  ../tmp_data/dino_lufkin


interactive(children=(IntSlider(value=0, description='_start_mydino', max=3), IntSlider(value=4, description='…

<function __main__.PrintMyFPolRoadandUsovLafkinFPolRoad(_start_mydino, _end_mydino, start_u_l_dino, end_u_l_din, title='Usov Data')>